<a href="https://colab.research.google.com/github/akimotolab/CMAES_Tutorial/blob/main/6_advanced_adaptation_mechanisms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# アドバンストな共分散行列の適応方法

これまでの議論では理解を簡単にするために，rank-$\mu$ update と呼ばれる，著者が最も簡単に理解できるであろうと考える共分散行列の更新方法のみに着目してアルゴリズムを紹介してきた．

最新のCMA-ES（dd-CMA-ES[1]）では，rank-one update[2]，rank-$\mu$ update[3]，active update[4]，diagonal acceleration[1]を組み合わせることで，より効率的な共分散行列の学習を可能としている．これまでに紹介してきた rank-$\mu$ updateを基準にした場合，それぞれのコンポーネントには以下のような利点がある：

* rank-one update：集団サイズが小さい場合に効率的．共分散行列のうち，固有値の大きな成分を学習する際に効率的．

* active update：共分散行列のうち，固有値の小さな成分を学習する際に効率的．

* diagonal acceleration：共分散行列の対角成分（分散行列）を学習する際に効率的．

ここでは，それぞれのコンポーネントについて紹介していく．

[1] Y. Akimoto, N. Hansen; Diagonal Acceleration for Covariance Matrix Adaptation Evolution Strategies. Evol Comput 2020; 28 (3): 405–435. doi: https://doi.org/10.1162/evco_a_00260

[2] N. Hansen and A. Ostermeier, "Completely Derandomized Self-Adaptation in Evolution Strategies," in Evolutionary Computation, vol. 9, no. 2, pp. 159-195, June 2001, doi: 10.1162/106365601750190398.

[3] Hansen N, Müller SD, Koumoutsakos P. Reducing the time complexity of the derandomized evolution strategy with covariance matrix adaptation (CMA-ES). Evol Comput. 2003 Spring;11(1):1-18. doi: 10.1162/106365603321828970. PMID: 12804094.

[4] G. A. Jastrebski and D. V. Arnold, "Improving Evolution Strategies through Active Covariance Matrix Adaptation," 2006 IEEE International Conference on Evolutionary Computation, Vancouver, BC, Canada, 2006, pp. 2814-2821, doi: 10.1109/CEC.2006.1688662.

#### 共分散行列の表現

以下，解生成分布の共分散行列$\Sigma$を
$$
\Sigma = \sigma^2 D C D
$$
と表現することにする．
$\sigma^2$はステップサイズであり，これまで通りCSAによって更新されるものとする．$D$は対角行列であり，$\sigma^2 D^2$が$\Sigma$の分散行列に対応する．すなわち，$\sigma [D]_{i,i}$が$i$番目の要素の標準偏差に対応する．$C$は対角成分が$1$となる行列であり，$\Sigma$の相関行列に対応する．Separable-CMAでは，$C = I$と固定し，$D$を更新していたと解釈できる．また，CMAでは$DCD$をまとめて更新していたと解釈できる．Diagonal accelerationでは，$D$と$C$を別々に更新することで効率的な学習を可能としている．以下ではこの分解を前提に，話を進めていく．

この共分散行列を用いた場合の解生成は以下の手順で行うことができる．
1. $z_i \sim \mathcal{N}(0, I)$
2. $y_i = \sqrt{C} z_i$
3. $x_i = m + \sigma D y_i$

#### Rank-$\mu$ update を用いた場合の更新式

通常のCMA-ES（$C$を更新する方法）とSeparable-CMA-ES（$D$を更新する方法）において，これまでに紹介してきたRank-$\mu$ updateを用いる場合の更新式をここに示す．Separable-CMA-ESの場合には，$D^2$を更新していくことになるが，ここではその変種として，$D$を更新していくバリエーションを考える．これは文献[1]で示されている方法である．

まず，$D$が固定されており（必ずしも$D = I$でなくてもよい），$C$のみを更新する場合，更新式は
$$
C \leftarrow C + c_\mu \sum_{i=1}^{\lambda} w_{i}\left( y_{i:\lambda}y_{i:\lambda}^\mathrm{T} - C \right)
$$
となる．ここで，rank-$\mu$ update を考える場合には，$w_1 \geq \dots \geq w_\lambda \geq 0$であり，$\sum_{i=1}^{\lambda} w_i = 1$を仮定していたことに注意されたい．

次に，$C = I$が固定されている場合の$D$の更新式は，
$$
\log D^2 \leftarrow \log D^2 + c_{\mu,D} \sum_{i=1}^{\lambda} w_{i}\mathrm{diag}\left( z_{i:\lambda}z_{i:\lambda}^\mathrm{T} - I \right)
$$
と表すことができる．指数関数を一次近似すれば，これまでに紹介したSeparable-CMAの更新式と対応していることが分かる．



## Rank-one update

まず，直感的な説明をする．Rank-$\mu$ update が各イテレーションでの$w_i > 0$となる$\mu$個の解の情報を活用して共分散行列を更新しているのに対し，rank-one update では複数イテレーションを通した解候補の情報を活用することを目的とする．そのため，$\mu$が小さい場合には，rank-one updateの方が情報をより効率的に活用できるようになる．また，rank-one updateではrank-$\mu$ updateでは活用できない符号情報を活用することが可能になる．これについては後述する．

更新式を見ていく．Rank-one update ではCSAでも利用されている進化パスを利用する．ただし，目的が異なるため，進化パスの定義が異なり，
$$
p_C \leftarrow (1 - c_C) p_C + \sqrt{\frac{c_C (2 - c_C)}{\sum_{j=1}^{\lambda} w_j^2} } \sum_{i=1}^{\lambda} w_i y_{i:\lambda}
$$
となる．
Separable-CMA-ESの場合，
$$
p_D \leftarrow (1 - c_D) p_D + \sqrt{\frac{c_D (2 - c_D)}{\sum_{j=1}^{\lambda} w_j^2} } \sum_{i=1}^{\lambda} w_i z_{i:\lambda}
$$
となり，こちらはCSAで利用されている進化パスと用いている累積率$c_D$が異なる点を除いて等しくなる．
これを用いて，$C$や$D$の更新式に，以下の要素を追加する：
$$
c_{1} (p_C p_C^\mathrm{T} - C) \\
c_{1,D} \mathrm{diag}(p_D p_D^\mathrm{T} - I)
$$

この更新式が不偏性を持つ，すなわち，追加した項がランダム関数のもとで期待値0となることは，CSAにおける不偏性の議論と同様にして導かれる．仮にランダムではなく，イテレーションを通して特定の方向の解が選択され続けているような場合，進化パスはそのような方向に対して大きくなるため，その方向への固有値が大きくなることが期待される．

先に述べた，符号情報を活用できる，というのは以下のようなことを意味する．ある解$y_1 = y$と$y_2 = -y$が与えられたとき，rank-$\mu$ updateでは
$$
w_1 (y_1 y_1^\mathrm{T} - C) + w_2 (y_2 y_2^\mathrm{T} - C) = (w_1 + w_2) (y y^\mathrm{T} - C)
$$
であり，$y_1 = y$と$y_2 = y$の場合にも
$$
w_1 (y_1 y_1^\mathrm{T} - C) + w_2 (y_2 y_2^\mathrm{T} - C) = (w_1 + w_2) (y y^\mathrm{T} - C)
$$
である．いずれの場合も，$y$方向に共分散行列の固有値を大きくする影響を持つ．
一方，rank-one updateでは，$y_1 = y$と$y_2 = -y$が与えられたとき，これらがまず進化パスとして累積されるため，この両者の情報は打ち消しあう．そのため，$y$方向に共分散行列の固有値を大きくする影響は持たない．他方，$y_1 = y$と$y_2 = y$が与えられた場合，進化パスが$y$方向に大きくなり，結果として共分散行列を$y$方向に大きくするような影響がある．

## Active update

## Diagonal Acceleration